In [1]:
import pandas as pd

import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm

import nltk
nltk.download('stopwords')

from spacy.lang.ru import Russian
import spacy
import ru_core_news_sm
nlp = spacy.load("ru_core_news_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gromo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_prosby = pd.read_excel('Просьбы жителей по благоустройству.xlsx')
df_prosby

,Адрес обращения,Дом,Дата,текст обращения
0,улица Советская,50А,2021-06-05 22:08:11.225,Тротуар возле рощи
1,улица Кукшумская,9,2021-06-11 20:56:03.802,прошу асфальтировать тротуар по улице Кукшумск...
2,улица Кукшумская,9,2021-06-11 21:00:13.376,прошу асфальтировать тротуар по улице Кукшумск...
3,улица Социалистическая,19,2021-06-12 13:08:16.467,"Бегают тараканы по всему подъезду, необходимо ..."
4,улица Ахазова,7,2021-06-15 17:10:23.232,Мы жильцы дома №7 живем с 1978 и капитального ...
...,...,...,...,...
8552,улица Композитора А.М.Токарева,4,2021-06-07 20:18:16.119,Озеленение территории микрорайона
8553,улица Эльменя,3,2021-06-04 14:14:31.632,Благоустройство детской площадки и парковки во...
8554,улица Мичмана Павлова,46,2021-06-08 21:26:49.098,Уборка мусора на территории дома 46. Новые ска...
8555,проспект Максима Горького,15,2021-06-08 12:37:29.409,Просим посодействовать в установке пандуса в п...


In [3]:
with open('bad_words.txt', 'r', encoding='utf-8') as filehandle:
    bad_words = filehandle.read()    

b_words = list(bad_words.split("', '"))

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

stopwords_list = stopwords.words('russian') + b_words

In [5]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    
    return ' '.join([w for w in tokens if w.isalpha() and len(w) >= 3])

def pos_cleaning(text):
    return [token.text for token in nlp(text) if token.pos_ == 'NOUN']   


def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    text = find_words(text)
    tokens = pos_cleaning(text)
    return list(lemmatize(tokens))

In [6]:
preprocessed_text = list(tqdm(map(preprocess, df_prosby['текст обращения'])))

0it [00:00, ?it/s]

In [7]:
df_prosby['text_clean'] = preprocessed_text

In [8]:
df_prosby.sample(5)

,Адрес обращения,Дом,Дата,текст обращения,text_clean
485,проспект Мира,88Б,2021-06-11 18:21:41.790,Установить защитный экран вдоль ж/д путей пере...,"[бордюр, тротуар, горка]"
3718,улица Хевешская,15 корп. 1,2021-06-13 19:54:26.005,осветить детскую площаку во дворе,"[площака, двор]"
1251,улица Строителей,20,2021-06-04 23:13:43.685,"Ремонт крыши, затопило квартиры",[]
6380,улица Энгельса,48,2021-06-14 20:34:10.124,очень просим сделать парковку,[парковка]
4600,улица Богдана Хмельницкого,10 корп. 1,2021-06-17 15:12:29.159,Огородить детскую площадку забором. Открыть ма...,[площадка]


In [9]:
compare_dict = {
    'зелёные насаждения': ['роща', 'дуб', 'кедр', 'клумба', 'ландшафт', 'берёза', 'лесопосадка',
                           'сирень', 'куст', 'ель', 'ёлочка', 'газон', 'клён', 'жасмина', 'цветок',
                           'рябина', 'посадка', 'зелень', 'полесадник', 'полисадник', 'озеленение',
                           'дубрава', 'дерьево', 'палисадник', 'растение', 'лесопарк', 'кустарник',
                           'лес', 'насаждение', 'питомец', 'цветник', 'озелениния', 'дерево', 'флора',
                           'пустырь', 'трава', 'заповедник', 'выгул', 'выгуливание', ],
    'асфальтовая дорожка': ['тротуар', 'дорога', 'бордюр','тратуар', 'дрога', 'асфальтирование',
                            'освальт', 'дорги', 'асвальт', 'асфальт', 'асфалта', 'кочка', 'ям',
                            'лужа', 'пандус', 'мост', 'бардюр', 'ямка', 'щебёнка', 'ухаб',
                            'сьезд', 'холм', 'подход', 'горка', 'переход', 'брусчатка', 'автодорога',
                            'выбоина', 'ливневка', 'отмосток', 'отмостка', 'обочина', 'песок', 'пандус',
                            'колесо', 'езда', 'бетон', 'зазор', 'дублёр', 'подъём', 'заезд', 'плитка', 'выезд',
                            'вьезд', 'вьезда', 'поворот', 'автомобиль', 'ходьба', ],
    'спортивная площадка': ['футбол', 'тренажёрка', 'баскетбол', 'спортивень', 'спорткомплекс',
                            'спортзал', 'спортинвентарь', 'спортплощадка', 'спортивна', 'фитнес',
                            'скейтпарк', 'скейтборд', 'спортцентр', 'каток', 'спорт', 'тренажёр', 'самокат',
                            'пеннибордовый', 'поле', ],
    'общественный туалет': ['туалет', 'санузел', ],
    'лавочка': ['скомейка', 'скамёк', 'лавочка', 'присесть', 'скаммейка', 'поставитьскомейка', 'лавка', 'скамейка', ],
    'освещение': ['светильник', 'освещение', 'фанари', 'темнота', 'свет', ],
    'беседка': ['беседка', ],
    'детский городок': ['городок', 'песочница', ],
    'навигация': ['знак', 'информаца', 'инф', ],
    'игровая площадка': ['мяч', 'игра', 'карусель', ' качель', 'скейтпарка', 'теннис', 'кочель', 'качели', ],
    'летний открытый кинотеатр': ['кинотеатр', 'сцена', ],
    'благоустройство': ['благоустройство', 'благоустроиство', 'благоустроитство', 'блогоустроиства',
                        'благоуствойство', 'благоустоиство', ' благоустроство', ' облагораживание',
                        'благоустройсть', 'улучшение', 'благоустроитва', 'реставрация', 'обустройство',
                        'мемориал', 'обелиск', 'преобразить', 'набережная', 'река', 'пляж', 'берег', 'водоём',
                        'ручей', 'речка', 'фонтан', 'парковка', 'стоянка', 'экопарковка', 'светафора', ],
    'парк': ['сквер', 'аллея', 'парк', ],
    'велосипедная дорожка': ['велодорожка', 'велосипедист', 'велосипед', 'велопрогулка', ],
    'место тихого отдыха': ['прогулка', 'бульвар', ],
    'беговая дорожка': ['бег', 'дорожка, '],
    'другое': ['ресторан', 'двор', 'стадион', 'рлощадка', 'площадк', 'плошадка', 'площака', 'развлечение',
               'полщадка', 'зона', 'площудка', 'плащадка', 'оращение', 'площадка', 'супермаркет', 'музей',
               'благосостояние', 'поляна', 'кафе', 'парковый', 'ролик', ]
}

In [10]:
def mapped_dict(text):
    map_d = {}
    for token in text:
        for key, value in compare_dict.items():
            if token in value:
                map_d[key] = 1
    return map_d

In [11]:
df_prosby['categories'] = df_prosby.text_clean.apply(mapped_dict)

In [12]:
df_prosby

,Адрес обращения,Дом,Дата,текст обращения,text_clean,categories
0,улица Советская,50А,2021-06-05 22:08:11.225,Тротуар возле рощи,"[тротуар, роща]","{'асфальтовая дорожка': 1, 'зелёные насаждения..."
1,улица Кукшумская,9,2021-06-11 20:56:03.802,прошу асфальтировать тротуар по улице Кукшумск...,[тротуар],{'асфальтовая дорожка': 1}
2,улица Кукшумская,9,2021-06-11 21:00:13.376,прошу асфальтировать тротуар по улице Кукшумск...,[тротуар],{'асфальтовая дорожка': 1}
3,улица Социалистическая,19,2021-06-12 13:08:16.467,"Бегают тараканы по всему подъезду, необходимо ...",[],{}
4,улица Ахазова,7,2021-06-15 17:10:23.232,Мы жильцы дома №7 живем с 1978 и капитального ...,"[двор, автомобиль, тротуар]","{'другое': 1, 'асфальтовая дорожка': 1}"
...,...,...,...,...,...,...
8552,улица Композитора А.М.Токарева,4,2021-06-07 20:18:16.119,Озеленение территории микрорайона,[озеленение],{'зелёные насаждения': 1}
8553,улица Эльменя,3,2021-06-04 14:14:31.632,Благоустройство детской площадки и парковки во...,"[благоустройство, площадка, парковка, двор]","{'благоустройство': 1, 'другое': 1}"
8554,улица Мичмана Павлова,46,2021-06-08 21:26:49.098,Уборка мусора на территории дома 46. Новые ска...,[скамейка],{'лавочка': 1}
8555,проспект Максима Горького,15,2021-06-08 12:37:29.409,Просим посодействовать в установке пандуса в п...,[пандус],{'асфальтовая дорожка': 1}
